In [1]:
import random

# Assinatura padrão

In [2]:
class Individuo:
    def __init__(self, grafo, fitness = 0, paredes_presentes = []):
        self.grafo = grafo #grafo que representa o individuo
        self.fitness = fitness#fitness do individuo
        self.paredes_presentes = paredes_presentes #paredes presentes nesse indiviuo, um subconjunto das paredes totais


In [3]:
populacao = []

In [4]:
vertices = ['a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a7', 'a8', 'a9', 'a10']

In [5]:
paredes_totais = [("a1", "a6"),("a6", "a7"),("a6", "a2"),("a7", "a8"),("a7", "a9"),("a7", "a5"),
("a5", "a9"),("a5", "a2"),("a5", "a4"),("a5", "a10"),("a9", "a10"),("a10", "a4"),("a4", "a2"), ("a4", "a3")]


In [6]:
def checar_individuo(individuo):
    "retorna True ou False se o individuo.grafo é conexo"
    pass


In [7]:
def init_pop():
    "devolve uma população gerada aleatoriamente de grafos conexos"
    pass


In [8]:
def fitness(individuo):
    "retorna o fitness do inviduo"
    pass


In [9]:
def mutacao(individuo):
    "retorna o novo individuo(mutado ou não)"
    pass


In [10]:
def selecao(populacao):
    "retorna os 100 melhores individuos"
    pass


# Implementação Cruzamento

In [11]:
random.seed()

In [12]:
grafo1 = {"a1":["a6"],
          "a2":["a6", "a5", "a4"],
          "a3":["a4"],
          "a4":["a10", "a2", "a3"],
          "a5":["a2"],
          "a6":["a1", "a2", "a7"],
          "a7":["a8", "a9", "a6"],
          "a8":["a7"],
          "a9":["a7"],
          "a10":["a4"]}
paredes_presentes = [("a4", "a2"), ("a1", "a6"), ("a6", "a2"), ("a5", "a2"), ("a4", "a3"),
                    ("a10", "a4"),("a6", "a7"),("a7", "a8"), ("a7", "a9") ]
individuo_1 = Individuo(grafo1, paredes_presentes = paredes_presentes)

In [13]:
grafo2 = {"a1":["a6"],
          "a2":["a6", "a5"],
          "a3":["a4"],
          "a4":["a10", "a3"],
          "a5":["a2", "a10", "a7"],
          "a6":["a1", "a2"],
          "a7":["a8", "a9", "a5"],
          "a8":["a7"],
          "a9":["a7"],
          "a10":["a4", "a5"]}
paredes_presentes = [ ("a1", "a6"), ("a6", "a2"), ("a5", "a2"), ("a4", "a3"),
                    ("a10", "a4"),("a7", "a8"), ("a7", "a9"), ("a7", "a5"), ("a5", "a10")]
individuo_2 = Individuo(grafo2, paredes_presentes = paredes_presentes)

In [14]:
def seleciona_pais(populacao):
    "seleciona os candidatos a pais"
    pass

In [15]:
def recombina_populacao(candidatos_pais):
    "Retorna os filhos criados pelos candidatos a pais"
    pass

In [57]:
def cruzamento(pai, mae):
    "Retorna o filho"
    global vertices
   
    vert_len = len(vertices)
    
    ponto = random.randint(0, vert_len)#escolhe um ponto de corte aleatorio
    
    #cria dois filhos vazios
    filho_1 = Individuo({"a1":[], "a2":[],"a3":[],"a4":[],"a5":[],"a6":[],"a7":[],"a8":[],"a9":[],"a10":[]})
    filho_2 = Individuo({"a1":[], "a2":[],"a3":[],"a4":[],"a5":[],"a6":[],"a7":[],"a8":[],"a9":[],"a10":[]})
    
    #aqui copia a primeira parte do gene dos genitores
    for i in range(0, ponto):
        #copiando gene do pai(filho_1) e da mãe(filho_2)
        # ----------------------------- filho_1 -----------------------------------
        #essas duas linhas servem para garantir que não haja elementos repetidos na lista de vizinhos do filho_1
        conjunto_1 = set(filho_1.grafo[vertices[i]])#conjunto de todos os vizinhos de vertices[i] no grafo filho_1
        conjunto_1.update(set(pai.grafo[vertices[i]]))#atualiza esse conjunto com os vizinhos de vertices[i] do pai
        
        filho_1.grafo[vertices[i]] = list(conjunto_1)#atribui o novo conjunto ao filho
        
        for element in conjunto_1:
            #esse loop é usado para atualizar a lista dos vizinhos com o vertices[i] no filho_1 e 2
            conj_1 = set(filho_1.grafo[element])
            conj_1.add(vertices[i])
            filho_1.grafo[element] = list(conj_1)
        
        # ----------------------------- filho_2 -----------------------------------
        #essas duas linhas servem para garantir que não haja elementos repetidos na lista de vizinhos do filho_2
        conjunto_2 = set(filho_2.grafo[vertices[i]])#conjunto de todos os vizinhos de vertices[i] no grafo filho_2
        conjunto_2.update(set(mae.grafo[vertices[i]]))#atualiza esse conjunto com os vizinhos de vertices[i] da mãe
        
        filho_2.grafo[vertices[i]] = list(conjunto_2)#atribui o novo conjunto ao filho_2
        
        for element in conjunto_2:  
            conj_2 = set(filho_2.grafo[element])
            conj_2.add(vertices[i])
            filho_2.grafo[element] = list(conj_2)
    
    #aqui copia a segunda parte do gene dos genitores
    for i in range(ponto, vert_len):
        #copiando gene do pai(filho_1) e da mãe(filho_2)
       
        # ------------------ filho_1 -------------------------------
        #essas duas linhas servem para garantir que não haja elementos repetidos na lista de vizinhos do filho_1
        conjunto_1 = set(filho_1.grafo[vertices[i]])#conjunto de todos os vizinhos de vertices[i] no grafo filho_1
        conjunto_1.update(set(mae.grafo[vertices[i]]))#atualiza esse conjunto com os vizinhos de vertices[i] da mãe
        
        filho_1.grafo[vertices[i]] = list(conjunto_1)#atribui o novo conjunto ao filho_1
        
        for element in conjunto_1:
            #esse loop é usado para atualizar a lista dos vizinhos com o vertices[i] no filho
            conj = set(filho_1.grafo[element])
            conj.add(vertices[i])
            filho_1.grafo[element] = list(conj)
       
        # ------------------- filho_2 -------------------------------
        conjunto_2 = set(filho_2.grafo[vertices[i]])#conjunto de todos os vizinhos de vertices[i] no grafo filho
        conjunto_2.update(set(pai.grafo[vertices[i]]))#atualiza esse conjunto com os vizinhos de vertices[i] do pai
        
        filho_2.grafo[vertices[i]] = list(conjunto_2)#atribui o novo conjunto ao filho
        
        for element in conjunto_2:    
            conj = set(filho_2.grafo[element])
            conj.add(vertices[i])
            filho_2.grafo[element] = list(conj)
            
    return [filho_1, filho_2]

In [58]:
a = cruzamento(individuo_1, individuo_2)

In [63]:
grafo3 = {"a1":["a6"],
          "a2":["a6", "a5", "a4"],
          "a3":["a4"],
          "a4":["a10", "a2", "a3", "a5"],
          "a5":["a2", "a4", "a10", "a9", "a7"],
          "a6":["a1", "a2", "a7"],
          "a7":["a8", "a9", "a6", "a5"],
          "a8":["a7"],
          "a9":["a7", "a5", "a10"],
          "a10":["a4", "a9", "a5"]}

individuo_3 = Individuo(grafo3)

In [64]:
grafo4 = {"a1":["a6"],
          "a2":["a6", "a5"],
          "a3":["a4"],
          "a4":["a3", "a5"],
          "a5":["a2", "a4"],
          "a6":["a1", "a2", "a7"],
          "a7":["a8", "a9", "a6"],
          "a8":["a7"],
          "a9":["a7", "a10"],
          "a10":["a9"]}

individuo_4 = Individuo(grafo4)

In [66]:
b = cruzamento(individuo_3, individuo_4)

In [68]:
b[0].grafo

{'a1': ['a6'],
 'a2': ['a4', 'a5', 'a6'],
 'a3': ['a4'],
 'a4': ['a10', 'a3', 'a5', 'a2'],
 'a5': ['a4', 'a9', 'a10', 'a7', 'a2'],
 'a6': ['a7', 'a1', 'a2'],
 'a7': ['a8', 'a9', 'a6', 'a5'],
 'a8': ['a7'],
 'a9': ['a10', 'a7', 'a5'],
 'a10': ['a4', 'a9', 'a5']}

In [69]:
b[1].grafo

{'a1': ['a6'],
 'a2': ['a5', 'a6'],
 'a3': ['a4'],
 'a4': ['a10', 'a3', 'a5'],
 'a5': ['a10', 'a4', 'a9', 'a2'],
 'a6': ['a7', 'a1', 'a2'],
 'a7': ['a8', 'a9', 'a6'],
 'a8': ['a7'],
 'a9': ['a10', 'a7', 'a5'],
 'a10': ['a4', 'a9', 'a5']}